In [4]:
from datetime import timedelta, datetime

import pandas as pd

from feast import (
    FeatureStore,
    Entity,
    FeatureView,
    Field,
    FileSource,
    Project,
    PushSource,
)
from feast.on_demand_feature_view import on_demand_feature_view
from feast.types import Float32, Float64, Int64, String, Array

ModuleNotFoundError: No module named 'feast'

In [ ]:
project = Project(name="intense_iguana", description="A project for documents retrieval")

In [ ]:
doc = Entity(name="item_id", join_keys=["item_id"])

In [ ]:
item_embed_df = pd.read_parquet(path='data/doc_embed.parquet')

In [ ]:
item_embed_push_fs = FileSource(
    name="document_embeddings",
    path="data/doc_embed.parquet",
    timestamp_field="event_timestamp",
)

In [ ]:
store = FeatureStore(repo_path=".")

In [ ]:
store.apply(project, doc, item_embed_push_fs)

In [ ]:
from feast.data_source import PushMode

In [ ]:
item_embed_push_source = PushSource(name='item_embed_push_source', batch_source=item_embed_push_fs)

In [ ]:
store.apply(item_embed_push_source)

In [ ]:
item_embedding_view = FeatureView(
    name="item_embedding",
    entities=[doc],
    ttl=timedelta(days=1),
    schema=[
        Field(name="item_id", dtype=Int64),
        Field(
            name="embedding",
            dtype=Array(Float32),
            vector_index=True,
            vector_search_metric="cosine",
        ),
    ],
    source=item_embed_push_source,
    online=True,
)

In [ ]:
store.apply(item_embedding_view)

In [ ]:
pushresult = store.push(push_source_name='item_embed_push_source', df=item_embed_df, to=PushMode.ONLINE)

In [ ]:
store.apply(doc)

In [ ]:
user_embed = [0.15] * 128

In [ ]:
print(store.retrieve_online_documents(query=user_embed, top_k=10, feature='item_embedding:item_id').to_df())